In [17]:
!pip show h5py

Name: h5py
Version: 3.12.1
Summary: Read and write HDF5 files from Python
Home-page: 
Author: 
Author-email: Andrew Collette <andrew.collette@gmail.com>
License: BSD-3-Clause
Location: /opt/anaconda3/envs/BRIDGE/lib/python3.9/site-packages
Requires: numpy
Required-by: 


In [14]:
import pandas as pd
import h5py
import numpy as np

class DSPrepper:

    def __init__(self, dataset_path: str, output_path: str):
        self.dataset_path = dataset_path
        self.output_path = output_path
        self.dataset = None

    def load_and_process(self):
        self.dataset = pd.read_csv(self.dataset_path)
        self._transform_encodings()
        self.dataset.to_csv(self.output_path)

    def _transform_encodings(self):
        def parse_encoding(x):
            try:
                return [float(value) for value in x.split(":")]
            except ValueError:
                return None
        self.dataset["global_encoding"] = self.dataset["global_encoding"].apply(parse_encoding)
        self.dataset.dropna(subset=["global_encoding"], inplace=True)
        self.dataset = self.dataset[["word_raw", "global_encoding"]]

    def convert_to_h5(self, h5_path: str):
        embeddings = np.array(self.dataset["global_encoding"].tolist(), dtype=np.float32)
        with h5py.File(h5_path, "w") as hf:
            hf.create_dataset("embeddings", data=embeddings)


In [16]:
prepper = DSPrepper(
    "/Users/giuliomartini/Desktop/FSU/BRIDGE/BRIDGE/models/giuliomartini_2024-10-24_14h15m37391ms_chkpt9999.csv", 
    "/Users/giuliomartini/Desktop/FSU/BRIDGE/changes/giuliomartini_2024-10-24_14h15m37391ms_chkpt9999_converted.csv"
    )
prepper.load_and_process()
prepper.convert_to_h5("/Users/giuliomartini/Desktop/FSU/BRIDGE/changes/giuliomartini_2024-10-24_14h15m37391ms_chkpt9999_converted.h5")